In [ ]:
from dotenv import load_dotenv
load_dotenv()

Translator demo: 

In [ ]:
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()
chat(
    [
        HumanMessage(
            content="Translate this sentence from English to French: I love programming."
        )
    ]
)

Joke generator demo: 

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

Club name generator demo: 

In [28]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

def generate_club_name(club_type):
    llm = OpenAI(temperature = 0.7)
    
    prompt_template_name = PromptTemplate(
        input_variables=['club_type'],
        template = "Suggest five cool names for a new {club_type} club. "
    )
    name_chain = LLMChain(llm=llm, prompt = prompt_template_name)
    
    response = name_chain({'club_type' : club_type})
    return response

if __name__ == "__main__":
    print(generate_club_name("Computer Engineering"))
    

{'club_type': 'Computer Engineering', 'text': '\n\n1. Code Crusaders\n2. Tech Titans\n3. Byte Brigade\n4. Circuit Savants\n5. Silicon Squad'}


The following is a demonstration on using vector stores with LanceDB to find text similarities. 
https://python.langchain.com/docs/get_started/quickstart
https://python.langchain.com/docs/modules/data_connection/vectorstores

In [30]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import LanceDB

import lancedb

db = lancedb.connect("/tmp/lancedb")
table = db.create_table(
    "my_table",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('art_of_war.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = LanceDB.from_documents(documents, OpenAIEmbeddings(), connection=table)

In [31]:
query = "What did Sun Tzu say about understanding the enemy?"
docs = db.similarity_search(query)
print(docs[0].page_content)

17. Thus we may know that there are five essentials for victory: (1)
He will win who knows when to fight and when not to fight. (2) He
will win who knows how to handle both superior and inferior forces.
(3) He will win whose army is animated by the same spirit throughout
all its ranks. (4) He will win who, prepared himself, waits to take
the enemy unprepared. (5) He will win who has military capacity and
is not interfered with by the sovereign. 

18. Hence the saying: If you know the enemy and know yourself, you
need not fear the result of a hundred battles. If you know yourself
but not the enemy, for every victory gained you will also suffer a
defeat. If you know neither the enemy nor yourself, you will succumb
in every battle. 

IV. Tactical Dispositions

1. Sun Tzu said: The good fighters of old first put themselves beyond
the possibility of defeat, and then waited for an opportunity of defeating
the enemy.


This is good, but does not give specific justification for its advice. 

In [34]:
query = "Sun Tzu's advice on applying to college?"
docs = db.similarity_search(query)
print(docs[0].page_content)

19. If it is to your advantage, make a forward move; if not, stay
where you are. 

20. Anger may in time change to gladness; vexation may be succeeded
by content. 

21. But a kingdom that has once been destroyed can never come again
into being; nor can the dead ever be brought back to life.

22. Hence the enlightened ruler is heedful, and the good general full
of caution. This is the way to keep a country at peace and an army
intact. 

XIII. The Use of Spies

1. Sun Tzu said: Raising a host of a hundred thousand men and marching
them great distances entails heavy loss on the people and a drain
on the resources of the State. The daily expenditure will amount to
a thousand ounces of silver. There will be commotion at home and abroad,
and men will drop down exhausted on the highways. As many as seven
hundred thousand families will be impeded in their labor.
